In [ ]:
import os
import glob
import torchvision
from tqdm.auto import tqdm, trange
import mmcv
from mmcv import VideoReader
from typing import List
from utils import makedirs_ifno, imshownp
import data

In [ ]:
input_folder = '/media/tormod/Den\ Lille/Thesis/videos/'
video_names = data.video_names
stripped_names = [v.split('.')[0] for v in video_names] # without .type
video_paths = [input_folder+v for v in video_names]
output_folder = '/media/tormod/Den Lille/Thesis/frames/'

In [10]:
vids = [VideoReader(f, cache_capacity=1) for f in video_paths]

In [ ]:
for frame in [0,2000,5000]:
    imshownp(vids[0][frame], bgr=True, savename=f'first_vid_{frame}.png')

In [11]:
def save_frames(vids: List[VideoReader]):
    for i, v in enumerate(vids):
        vname = stripped_names[i]
        print(f'Processing {vname}...')
        video_folder = f'{output_folder}{vname}/'
        makedirs_ifno([video_folder])
        for j, f in enumerate(tqdm(v)):
            mmcv.imwrite(f, f'{video_folder}{j:06d}.png')

In [12]:
save_frames(vids)

Processing IMG_2320...


# Join MMPose predictions

In [33]:
def load_mmpose_data(path, stop=None):
    files = os.listdir(path)
    files.sort(key=lambda s: int(s.split('.')[0]))
    no_files = stop if stop else len(files)
    files = files[:no_files]
    data = np.zeros((no_files, 17, 3))
    for i, f in tqdm(enumerate(files), total=no_files):
        kp_frame = np.load(path + f, allow_pickle=True)
        if len(kp_frame) > 0:
            kp_frame = kp_frame[0]['keypoints']
            data[i] = kp_frame
    return data

In [54]:
in_folder = '/home/tormod/ucph-erda-home/mmpose_results'
out_folder = '/home/tormod/ucph-erda-home/mmpose_anno'
makedirs_ifno([out_folder+'/'])
for name in data.stripped_names:
    mmpose_data = load_mmpose_data(f'{in_folder}/{name}/')
    np.save(f'{out_folder}/{name}.npy', mmpose_data)

In [ ]:
np.load('/home/tormod/ucph-erda-home/IMG_2139.npy')